In [4]:
import sys
sys.path.append('../')
import Phase_Detector.Utility as Utility
import torch

In [5]:
from torch.nn import CrossEntropyLoss
import torch
loss = CrossEntropyLoss()
print(loss)

CrossEntropyLoss()


In [48]:
y_true = [1,0,0,1,0,0,1,0,0,]
y_pre_tensor = [torch.tensor([0.1,0.8]).reshape(1,2) for i in range(len(y_true))]
metric = Utility.Metrics(2,y_pred=y_pre_tensor, y_true_label=y_true)
#metric.get_accuracy()

Test Dataloader

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

# 定义一个简单的自定义数据集
# 示例数据
data = [i for i in range(10)]  # 一个包含0-9的简单列表

# 创建Dataset
simple_dataset = ImageDataset(data)

# 创建DataLoader
#data_loader = DataLoader(simple_dataset, batch_size=2, shuffle=True)

# 迭代DataLoader
#for batch in data_loader:
 #   print(batch)


c:\Users\098986\AppData\Local\anaconda3\envs\CILM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from torch import tensor
import torch
import nibabel as nib

from torch.utils.data import WeightedRandomSampler

import numpy as np
import pandas as pd
import os
from monai.transforms import Compose

from monai.data import ImageDataset,DataLoader


class Image_Dataset(ImageDataset):
    def __init__(self,image_files,labels,transform_methods=None,data_aug=True,label_name=None,*args,**kwargs):
        """
        args:
            image_files: list of image files path
            labels: list of labels
            transform_methods: list of transform methods
            data_aug: True if data augmentation is used
            label_name: name of the label
        
        """
        if data_aug:
            transform = Compose(transform_methods)
        else:
            transform = None

        super().__init__(image_files=image_files,labels=labels,transform=transform,*args, **kwargs)

    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self,index):
        image = self.image_files[index]
        label = self.labels[index]

        # get image array
        image = nib.load(image).get_fdata()
        # here to do windowing

        image = tensor(image)
        return image,label
    
class Data_Loader(DataLoader):
    def __init__(self,dataset,batch_size,num_workers=0,*args,**kwargs):
        super().__init__(dataset=dataset,batch_size=batch_size,num_workers=num_workers,*args,**kwargs)
    
    
    def build_train_loader(self):
        return DataLoader(self.dataset,batch_size=self.batch_size,shuffle=True,num_workers=self.num_workers,drop_last=True,*self.args,**self.kwargs)

    def build_vali_loader(self):
        return DataLoader(self.dataset,batch_size=self.batch_size,shuffle=False,num_workers=self.num_workers,drop_last=False,*self.args,**self.kwargs)
    
    def build_test_loader(self):
        return DataLoader(self.dataset,batch_size=self.batch_size,shuffle=False,num_workers=self.num_workers,drop_last=False,*self.args,**self.kwargs)


In [11]:

import monai
import torch

model = monai.networks.nets.resnet10()
model.parameters()

<generator object Module.parameters at 0x00000186E6371F20>

In [33]:
def build_optimizer(parameters,**kwargs):
    print(kwargs)
    return torch.optim.Adam(parameters,**kwargs)


optimizer_param = {"lr":0.001}
optimizer = build_optimizer(model.parameters(),**optimizer_param)

{'lr': 0.001}


In [46]:
from torch.nn import CrossEntropyLoss

class Loss:
    def __init__(self,*args,**kwargs):
        """
        args:
            args only have one loss function
        """
        self.args = args

    def build_loss(self):
        return self.args[0]
    
    def calculate_loss(self,*args):
        pass


CrossEntropyLoss()

In [20]:
from monai.transforms import Compose,EnsureChannelFirst,RandZoom,RandRotate,RandFlip,ToTensor
import inspect
transform_param = {"transform_methods":[
                                EnsureChannelFirst(),
                                # Data augmentation
                                RandZoom(prob = 0.5, min_zoom=1.0, max_zoom=1.2),
                                RandRotate(range_z = 0.35, prob = 0.8),
                                RandFlip(prob = 0.5),
                                # To tensor
                                ToTensor()
                                ]}
transform_param['transform_methods'][1].__dict__


{'_do_transform': True,
 'prob': 0.5,
 '_lazy': False,
 'min_zoom': (1.0,),
 'max_zoom': (1.2,),
 'mode': area,
 'padding_mode': edge,
 'align_corners': None,
 'dtype': torch.float32,
 'keep_size': True,
 'kwargs': {},
 '_zoom': [1.0]}

Test Dataloader


In [8]:
from sklearn.metrics import accuracy_score
import torch
a = torch.tensor([0,1])
b = [0,]
accuracy_score(a,b)

0.5

In [14]:
a = Data_Loader(dataset=dataset,batch_size=1,num_workers=0).build_train_loader()
len(a)

40

# Binary classifaction


In [2]:
import pandas as pd
pvp_label = pd.read_csv('../../Data/CT_Phase/True_Label/Phase_PVP.csv')
#将Phase中的Phase的1和0都换成0,2换成1
pvp_label['Phase'] = pvp_label['Phase'].apply(lambda x: 0 if x == 1 or x==0 else 1)


In [6]:
pvp_label.to_csv('../../Data/CT_Phase/True_Label/Phase_PVP.csv',index=False)




METRICS



In [29]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from monai.metrics import get_confusion_matrix,compute_roc_auc
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, f1_score
from torch import tensor
from sklearn.metrics import precision_score, recall_score
import torch
import pandas as pd
import numpy as np
class Metrics():
    def __init__(self,num_class=2,y_pred=None,y_true_label=None,targets=None):
        """
        args:
            y_pred: list of predicted tensor
            y_true_label: list of true labels
            targets: dicts of targets and their labels
        """
        self.num_class = num_class
        self.four_rate_dic = {str(i):{'tp':0,'fp':0,'tn':0,'fn':0} for i in range(num_class)}
        self.y_true_label = np.array(y_true_label)
        self.y_pred_label = [torch.argmax(y_pre,dim=1).detach().cpu().numpy().tolist() for y_pre in y_pred]
        self.y_pred_label = [item for sublist in self.y_pred_label for item in sublist]

        self.y_pred_label = np.array(self.y_pred_label)
        self.y_pred_one_hot = torch.nn.functional.one_hot(torch.tensor(self.y_pred_label,dtype=torch.int64),num_classes=self.num_class)
        self.y_true_one_hot = torch.nn.functional.one_hot(torch.tensor(self.y_true_label.tolist(),dtype=torch.int64),num_classes=self.num_class)
    
    def calculate_metrics(self):
        self.metrics = {str(i): {'f1': 0, 'auc': 0, 'accuracy': 0, 'precision': 0, 'recall': 0} for i in range(self.num_class)}

        for i in range(self.num_class):
            true_binary = (self.y_true_label == i).astype(int)
            pred_binary = (self.y_pred_label == i).astype(int)
            print(true_binary,pred_binary,666)
            self.metrics[str(i)]['f1'] = f1_score(true_binary, pred_binary)
            self.metrics[str(i)]['precision'] = precision_score(true_binary, pred_binary)
            self.metrics[str(i)]['recall'] = recall_score(true_binary, pred_binary)

            if len(np.unique(true_binary)) > 1:
                self.metrics[str(i)]['auc'] = roc_auc_score(true_binary, self.y_pred_one_hot[:, i])

            self.metrics[str(i)]['accuracy'] = accuracy_score(true_binary, pred_binary)

        return self.metrics



    def get_roc(self,average='macro'):
        return compute_roc_auc(self.y_pred_one_hot,self.y_true_one_hot,average)
        

    def get_four_rate(self) -> tensor:
        """
        args:
            y_pred: (B,C) one-hot tensor
            y_true: (B,C) one-hot tensor
        """
        confu_matrix = get_confusion_matrix(self.y_pred_one_hot,self.y_true_one_hot)
        #calculate tp,fp,tn,fn
        for i in range(self.num_class):
            self.four_rate_dic[str(i)]['tp'] += confu_matrix[:,i,0].sum()
            self.four_rate_dic[str(i)]['fp'] += confu_matrix[:,i,1].sum() 
            self.four_rate_dic[str(i)]['tn'] += confu_matrix[:,i,2].sum() 
            self.four_rate_dic[str(i)]['fn'] += confu_matrix[:,i,3].sum()
        return self.four_rate_dic
    
    def get_accuracy(self) -> float:
        """
        args:
            y_pred_label: list of predicted labels
            y_true_label: list of true labels
        """
        accuracy = accuracy_score(self.y_pred_label,self.y_true_label)
        return accuracy
    
    def get_f1_score(self,average='macro') -> float:
        return f1_score(self.y_pred_label,self.y_true_label,average=average)
    

    def generate_metrics_df(self,epoch):

        # 将指标数据整理成列表形式
        metrics_data = []
        for class_id, class_metrics in self.metrics.items():
            data_row = {"epoch": epoch}  # 首先添加 epoch
            data_row.update({"class_id": class_id})  # 然后添加 class_id
            data_row.update(class_metrics)  # 最后添加其他指标
            metrics_data.append(data_row)



        # 创建DataFrame
        df = pd.DataFrame(metrics_data)

        return df


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义网络结构
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 参数设置
input_size = 784  # 例如，28x28的图像展平后的大小
hidden_size = 100  # 隐藏层的大小
num_classes = 10   # 类别数，例如MNIST手写数字识别为10

# 创建网络
model = SimpleNet(input_size, hidden_size, num_classes)

# 假设我们有一些随机数据作为输入
batch_size = 2 # 批量大小
x = torch.randn(batch_size, input_size)  # 生成随机数据

# 前向传播
output = model(x)


# 输出结果
print(output.shape)  # 应该是 [batch_size, num_classes]

metrics = Metrics(10,y_pred=[output],y_true_label=[1 for i in range(2)])
metrics.calculate_metrics()

torch.Size([2, 10])
[0 0] [0 0] 666
[1 1] [0 0] 666
[0 0] [0 0] 666
[0 0] [0 0] 666
[0 0] [0 0] 666
[0 0] [0 0] 666
[0 0] [0 0] 666
[0 0] [0 0] 666
[0 0] [1 1] 666
[0 0] [0 0] 666


c:\Users\098986\AppData\Local\anaconda3\envs\CILM\lib\site-packages\sklearn\metrics\_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\098986\AppData\Local\anaconda3\envs\CILM\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\098986\AppData\Local\anaconda3\envs\CILM\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\098986\AppData\Loc

{'0': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '1': {'f1': 0.0, 'auc': 0, 'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0},
 '2': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '3': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '4': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '5': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '6': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '7': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0},
 '8': {'f1': 0.0, 'auc': 0, 'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0},
 '9': {'f1': 0.0, 'auc': 0, 'accuracy': 1.0, 'precision': 0.0, 'recall': 0.0}}

,epoch,class_id,f1,auc,accuracy,precision,recall
0,1,0,0.0,0,1.0,0.0,0.0
1,1,1,0.0,0,0.0,0.0,0.0
2,1,2,0.0,0,1.0,0.0,0.0
3,1,3,0.0,0,1.0,0.0,0.0
4,1,4,0.0,0,1.0,0.0,0.0
5,1,5,0.0,0,1.0,0.0,0.0
6,1,6,0.0,0,1.0,0.0,0.0
7,1,7,0.0,0,1.0,0.0,0.0
8,1,8,0.0,0,0.0,0.0,0.0
9,1,9,0.0,0,1.0,0.0,0.0


In [13]:
output

tensor([[-0.0618, -0.0941,  0.1946, -0.2964,  0.0383,  0.0465, -0.2030, -0.2793,
          0.1523, -0.2352]], grad_fn=<AddmmBackward0>)

In [7]:
y_true_label=[1 for i in range(10)]
y_true_label 

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]